## Actividad: Uso de claves asimétricas

En esta actividad se comparte un mensaje cifrado por medio de una clave asimétrica. Si bien en este cuaderno se hace la codificación y la decodificación, es necesario que los estudiantes compartan los mensajes y la clave pública para que entiendan la dinámica de funcionamiento del par de claves privada y pública

Se recomienda que trabajen en grupos que compartirán los mensajes y las claves públicas


In [11]:
# Importar las bibliotecas necesarias
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes


In [12]:
# Generar un par de claves (privada y pública)
def generar_claves_asimetricas():
    """Genera y guarda un par de claves asimétricas en archivos."""
    clave_privada = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    clave_publica = clave_privada.public_key()

    # Guardar clave privada
    with open("clave_privada.pem", "wb") as clave_privada_archivo:
        clave_privada_archivo.write(
            clave_privada.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,
                encryption_algorithm=serialization.NoEncryption()
            )
        )

    # Guardar clave pública
    with open("clave_publica.pem", "wb") as clave_publica_archivo:
        clave_publica_archivo.write(
            clave_publica.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
        )

    print("Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.")

# Cargar clave privada
def cargar_clave_privada():
    """Carga la clave privada desde un archivo."""
    with open("clave_privada.pem", "rb") as clave_privada_archivo:
        return serialization.load_pem_private_key(
            clave_privada_archivo.read(),
            password=None
        )

# Cargar clave pública
def cargar_clave_publica():
    """Carga la clave pública desde un archivo."""
    with open("clave_publica.pem", "rb") as clave_publica_archivo:
        return serialization.load_pem_public_key(
            clave_publica_archivo.read()
        )

# Cifrar un mensaje asimétrico
def cifrar_mensaje_asimetrico(mensaje, clave_publica):
    """Cifra un mensaje utilizando la clave pública."""
    mensaje_cifrado = clave_publica.encrypt(
        mensaje.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return mensaje_cifrado

# Descifrar un mensaje asimétrico
def descifrar_mensaje_asimetrico(mensaje_cifrado, clave_privada):
    """Descifra un mensaje cifrado utilizando la clave privada."""
    mensaje_descifrado = clave_privada.decrypt(
        mensaje_cifrado,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return mensaje_descifrado.decode()

# Validar la autenticidad del remitente
def validar_autenticidad(mensaje, firma, clave_publica):
    """Valida que el mensaje proviene del remitente verificando la firma con la clave pública."""
    try:
        clave_publica.verify(
            firma,
            mensaje.encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception:
        return False

# Firmar un mensaje
def firmar_mensaje(mensaje, clave_privada):
    """Genera una firma para el mensaje utilizando la clave privada."""
    firma = clave_privada.sign(
        mensaje.encode(),
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return firma


mensaje = "Una máquina simple es un dispositivo mecánico que cambia la dirección o la magnitud de una fuerza"

generar_claves_asimetricas()
clave_publica = cargar_clave_publica()
clave_privada = cargar_clave_privada()
mensaje_cifrado_asimetrico = cifrar_mensaje_asimetrico(mensaje, clave_publica)
print(f"Mensaje cifrado (asimétrico): {mensaje_cifrado_asimetrico}")
mensaje_descifrado_asimetrico = descifrar_mensaje_asimetrico(mensaje_cifrado_asimetrico, clave_privada)
print(f"Mensaje descifrado (asimétrico): {mensaje_descifrado_asimetrico}")


In [13]:

# Parte 3: Validación de Autenticidad firmando con la clave privada
print("\n[Validación de Autenticidad]")
mensaje = "Este es un mensaje de prueba"
generar_claves_asimetricas()
# Cargar claves desde los archivos
clave_privada = cargar_clave_privada()
clave_publica = cargar_clave_publica()

firma = firmar_mensaje(mensaje, clave_privada)
print(f"Firma generada: {firma}")
es_valido = validar_autenticidad(mensaje, firma, clave_publica)
print(f"¿El mensaje es auténtico? {'Sí' if es_valido else 'No'}")



[Validación de Autenticidad]
Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.
Firma generada: b'S\xc6\xc1\xa4\xbdU\x90\xbeU[\x19bk\x04U\xbb\x95U1]\xdc`\xdf\x8b\nOH\xdd\xd1\xd2\xf9\xbb-u\xb7\xa3xt\x8f\xe5A\xf4b\xd2C#\xe1f\xd9\xe9bw\x05Gx~,\x82\x98\xd0\xbb=\x14K{?\x98\xcb\x92W\xc4\xd0rd\xeb\x93\xecR\x90\xe9\xe1\xbc\xb4\x0f\x9f\xf1o5\xc5>\xdf\xac\x96\x86\xcd\x0bp\x00V\x89\xf4\xc6\x12\x94\xa4\x93\x17\x84\xe0\xda\x88-\xbf6:(\xe2\xbd\xce:v\x01\xd7\xc7\xcd\x0c\':0b\xbe\xf0\xec1\x12\x86\x8eI7\xf7\x12\x98<>$\xb35N\xda%`)\xb6\x19\x80\x1eM\xea\xb18\x13\xc5\xfeZ\x93\xcd\xdb\xde\xd9\x87\x99\x96\xca\xdf\xad\xd5FZ\xadk\xfe%V%\xf8"\x8d\xabg^c\x88"JZ\xe7!e\xac\xa0Eg\x92\xf9P\xe0\x8b\xfc\xe9\xb0\xaa(\xd8\x0f\xaa|\xb6\xfc\x03\\qB\xc3\x9d\x06#j\x15\xd1~\xec\xd5a]\xba\xefV\x97d\xc5A"\xe8\x9f9\x0fj\x1f\xa4\xdcP\xdd\x12\'\x8b\xf2'
¿El mensaje es auténtico? Sí


In [14]:
# Modificación del mensaje original
print("\n1. Prueba con mensaje modificado:")
mensaje_original = "Este es un mensaje de prueba"
mensaje_modificado = "Este es un mensaje de prueba MODIFICADO"

# Usar la firma del mensaje original con el mensaje modificado
firma_original = firmar_mensaje(mensaje_original, clave_privada)
es_valido_modificado = validar_autenticidad(mensaje_modificado, firma_original, clave_publica)
print(f"Mensaje original: '{mensaje_original}'")
print(f"Mensaje modificado: '{mensaje_modificado}'")
print(f"¿El mensaje modificado es auténtico con la firma original? {'Sí' if es_valido_modificado else 'No'}")



1. Prueba con mensaje modificado:
Mensaje original: 'Este es un mensaje de prueba'
Mensaje modificado: 'Este es un mensaje de prueba MODIFICADO'
¿El mensaje modificado es auténtico con la firma original? No


### Actividad de reflexión
Discutir con los estudiantes:
- ¿Qué ventajas tiene el cifrado asimétrico?
 * Permite un intercambio más seguro ya que mantiene secreta la clave privada de un sólo lado.   
- ¿Qué posibles fallas de seguridad pueden existir?
 *  Si alguien obtiene la clave privada puede obtener acceso a nuestra información y también suplantarnos.
- ¿Qué sucede si yo pierdo mi clave privada? ¿Qué sucede si la comparto?
 * Pierdo acceso a la información cifrada y un tercero puede suplantarme, esta clave es única he intrasferible.
- Intente codificar la fábula de la lechera u otro texto largo ¿Por qué da un error? ¿Cómo se puede arreglar este error para compartir mensajes cifrados?
* Fue necesario agregar las lineas de creación de clave
- Comparta mensajes largos cifrados entre los estudiantes y que ellos validen la autencididad del remitente